In [4]:
# Classify CK+ moderate images into ModEm folder
import os

for root, dirs, files in os.walk("./cohn-kanade-images2"):
    filelist=[]
    for name in files:
        filename = os.path.join(root, name)
        if '.png' in filename:
            filelist.append(filename)
    if len(filelist) > 0:
        modfile = filelist[int(len(filelist)/2)]
        !cp $modfile ModEm
        print

In [ ]:
# Get numpy array of expression and neutral differences for each image (strong)
import cv2
import dlib
import os
import numpy as np
import math

# Initialize variables for dLib
predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
# Initialize other variables
NLA = []
ELA = []
tempLDA = []

# Get landmarks for neutral images
for root, dirs, files in os.walk("./Neutral"):
    for name in files:
        filename = os.path.join(root, name)
        # Read image if of correct type (8bit gray or RGB)
        if '.png' in filename:
            image = cv2.imread(filename, 1)
            # Detect faces and initialize variables for next step
            dets = detector(image, 1)
            # Get list of x-coordinates and y-coordinates of facial landmarks for each face
            for d in dets:
                imageNA = []
                shape = predictor(image, d)
                for index in range(0, 68):
                    point = shape.part(index)
                    tuplepoint = (point.x, point.y)
                    imageNA.append(tuplepoint)
                NLA.append(imageNA)
                
# Get landmarks for expression images
for root, dirs, files in os.walk("./Expression"):
    for name in files:
        filename = os.path.join(root, name)

        # Read image if of correct type (8bit gray or RGB)
        if '.png' in filename:
            image = cv2.imread(filename, 1)

            # Detect faces and initialize variables for next step
            dets = detector(image, 1)

            # Get list of x-coordinates and y-coordinates of facial landmarks for each face
            for d in dets:
                imageEA = []
                shape = predictor(image, d)
                for index in range(0, 68):
                    point = shape.part(index)
                    tuplepoint = (point.x, point.y)
                    imageEA.append(tuplepoint)
                ELA.append(imageEA)

# Get list of Euclidean differences for each landmark point for each image, convert list to numpy.arrray, add to big list
for img in range(len(NLA)):
    tempImgLDA = []
    for point in range(68):
        dif = math.sqrt((math.pow(ELA[img][point][0]-NLA[img][point][0], 2)) + (math.pow(ELA[img][point][1]-NLA[img][point][1], 2)))
        tempImgLDA.append(dif)
    imgLDA = np.asarray(tempImgLDA)
    tempLDA.append(imgLDA)

# Convert big list of all differences for all images to numpy.array
LDA = np.asarray(tempLDA)

# Subtract the average from the LDA values to see if there is any change
for img in range(len(LDA)):
    sum = 0
    for valueIndex in range(len(LDA[img])):
        sum+=LDA[img][valueIndex]
    average = sum/68
    for vI in range(len(LDA[img])):
        LDA[img][vI]-=average
        

In [1]:
# WORKING VERSION OF COMBINED DRAFTS-> STRONG EMOTION CLASSIFIERS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate normalized Euclidean Differences array
# Generate target values array
# Feed to SVM, evaulate with Cross Validation
import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    for d in dets:
        shape = predictor(image, d)
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)

"""MAIN PROGRAM BEGINS"""
# Get paths to face images and emotion labels
imagedir = []
labeldir = []
root, dirs, files = os.walk("./cohn-kanade-images",).next()
for d in dirs:
    imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))

# initialize variables
dataset = []
NLA = []
ELA = []

# look over all directories of individuals S001 through S999
for d in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images',d)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images',d,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            faceimage = os.path.join(curdir,files[-1])

            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',d,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()
                    
                    # append label to dataset
                    dataset.append([baseimage, faceimage, targetlabel])

"""68-POINT LANDMARK DETECTION MODEL"""
# Get array of Euclidean differences between neutral and expression 68-landmark arrays
tempLDA = []
for imgIndex in range(len(dataset)):
    nImgName = dataset[imgIndex][0]
    eImgName = dataset[imgIndex][1]
    nLA = get68LMA(nImgName)
    eLA = get68LMA(eImgName)
    NLA.append(nLA)
    ELA.append(eLA)
    tempLDA.append(getEucDist(nLA, eLA))
LDA = np.asmatrix(tempLDA)

# Create target value array for expression images
tempTargetValues = []
for imgSet in range(len(dataset)):
    tempTargetValues.append(dataset[imgSet][2])
targetValues = np.asarray(tempTargetValues)

# Feed LDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train, x_test, y_train, y_test = cross_validation.train_test_split(LDA, targetValues, test_size=0.4, random_state=2)
normCLF = svm.SVC(kernel='linear').fit(x_train, y_train)
predictions = normCLF.predict(x_test)
cmatrix = confusion_matrix(y_test, predictions)

# Format CMatrix to show percentages, rounded to 1 decimal point
np.set_printoptions(precision=1)
rowSums = np.sum(cmatrix, axis=1)
pCMatrix = np.divide(cmatrix*100., rowSums, dtype=float)
print pCMatrix


"""PHYSIOLOGICALLY BASED LANDMARK DETECTION MODEL"""
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)
np.savetxt('PDAfile', PDA) #Save PDA as human-readable format file

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='linear').fit(x_train2, y_train2)
predictions2 = physCLF.predict(x_test2)
cMatrix2 = confusion_matrix(y_test2, predictions2)

# Format CMatrix2 to show percentages, already rounded to 1 decimal point
rowSums2 = np.sum(cMatrix2, axis=1)
pCMatrix2 = np.divide(cMatrix2*100., rowSums2, dtype=float)
print pCMatrix2

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-d64e5218f437>, line 143)